# PHM North America challenge '23

# 03 - Offline vibration fingerprint extraction

This notebook constructs the context-sensitive vibration fingerprints from the vibration data that was previously preprocessed in [02_data_processing.ipynb](02_data_processing.ipynb).

In [ ]:
%load_ext autoreload
%autoreload 2

from conscious_engie_icare.data.phm_data_handler import CACHING_FOLDER_NAME, FPATH_DF_V_TRAIN_FOLDS, \
                                                        FPATH_META_DATA_TRAIN_FOLDS, FPATH_META_DATA_TRAIN_FOLDS, \
                                                        fetch_and_unzip_data
from conscious_engie_icare.nmf_profiling import extract_nmf_per_number_of_component
from conscious_engie_icare.viz.viz import illustrate_nmf_components_for_paper
from conscious_engie_icare import distance_metrics
import pickle
import os
from tqdm import tqdm
import glob
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist
import string
from matplotlib.colors import LogNorm

# ignore convergence warnings (1000 iterations reached by NMF)
import warnings;
warnings.filterwarnings('ignore');

Set `CACHE_RESULTS` to `True` to cache the results of the feature extraction process. This will speed up the notebook execution time in subsequent executions.

In [ ]:
CACHE_RESULTS = True

Load [previously preprocessed](02_data_processing.ipynb) vibration data.

In [ ]:
fetch_and_unzip_data()

with open(FPATH_DF_V_TRAIN_FOLDS, 'rb') as file:
    df_V_train_folds = pickle.load(file)

with open(FPATH_META_DATA_TRAIN_FOLDS, 'rb') as file:
    meta_data_train_folds = pickle.load(file)

# extract list of frequency band columns for later usage
cols_ = df_V_train_folds[0].columns
BAND_COLS = cols_[cols_.str.contains('band')].tolist()

## Non-negative matrix factorization (NMF)

Below, we decompose the vibration decomposition matrix $\mathbf{V}$ into a weight matrix $\mathbf{W}$ and a component matrix $\mathbf{H}$ for each of the 100 folds.
We decompose the matrix into up to 40 components, i.e. there are around 4000 different non-negative matrix factorizations performed.
As the cell takes around 20 minutes to execute, we cache the results to avoid re-executing it in subsequent runs.

In [ ]:
RECOMPUTE = False        # only set to True, if NMF should be recomputed
MAX_N_COMPONENTS = 40    # maximum number of components used to recompute

for fold, df_V_train_ in enumerate(tqdm(df_V_train_folds, desc='Extracting NMF models per fold')):
    fpath = os.path.join(CACHING_FOLDER_NAME, 'df_nmf_models_folds', f'df_nmf_models_folds_{fold}.pkl')
    if not os.path.exists(fpath) or RECOMPUTE:
        df_nmf_models_ = extract_nmf_per_number_of_component(
            df_V_train_, n_components=MAX_N_COMPONENTS, timestamps=df_V_train_.index, verbose=False
        )
        if CACHE_RESULTS:
            os.makedirs(os.path.dirname(fpath), exist_ok=True)
            pickle.dump(df_nmf_models_, open(fpath, 'wb'))

# load data from disk
fpaths = glob.glob(os.path.join(CACHING_FOLDER_NAME, 'df_nmf_models_folds', f'df_nmf_models_folds_*.pkl'))
# sort in ascending order based on the integer value <i> in df_nmf_models_folds_<i>.pkl
fpaths = sorted(fpaths, key=lambda x: int(x.split('_')[-1].split('.')[0]))
df_nmf_models_folds = [pickle.load(open(fpath, 'rb')) for fpath in tqdm(fpaths, desc='Loading folds from disk')]

Below, we illustrate the decomposition of the vibration data for the first fold.

- The left plot shows the decomposition matrix $\mathbf{V}$ as it was preprocessed in the previous notebook. Each vibration measurement is represented as a row in the matrix, each column represents an order transformed and normalized bin. We stack the 3 different measurement directions on top of each other, resulting in 645 vibration measurements, corresponding to the 215 measurements in the training set.
- The middle two plots visualize how well the decomposition matrix $\mathbf{V}$ can be approximated. 
    - The top plot illustrates the cumulative explained variance of a principal component analysis (PCA) of the decomposition matrix $\mathbf{V}$. It serves as an indication of an upper bound for how well the signal could be expressed using PCA.
    - The bottom plot illustrates the reconstruction error of the decomposition matrix $\mathbf{V}$ using the NMF components. The reconstruction error is calculated as the Frobenius norm of the difference between the original matrix and the reconstructed matrix.
- The right plot illustrates the component matrix $\mathbf{H}$. Each individual component expresses a different peak. The number of components is determined by how many components in a in parallel running PCA are needed to explain 95% of the variance in the data, corresponding to 5 components in this case.

In [ ]:
# plot hyperparameters for NMF
FOLD = 2
QMIN = 0.001
QMAX = 0.999
MIN_EXPLAINED_VARIANCE = 95
ORDER_COMPONENTS = None # don't order the components

df_V_train_ = df_V_train_folds[FOLD]
df_nmf_models_ = df_nmf_models_folds[FOLD]

# get minimum and maximum for feature space
vmin = df_V_train_.stack().quantile(q=QMIN)
print(f'    - took {vmin} (=0.01 quantile) as vmin for plotting feature space')
vmax = df_V_train_.stack().quantile(q=QMAX)
print(f'    - took {vmax} (=0.99 quantile) as vmax for plotting feature space')

# calculate explained variance
print(f'- calculating explained variance...')
pca = PCA(n_components=len(BAND_COLS), random_state=42)
pca.fit(df_V_train_)
explained_variance_ratio = pca.explained_variance_ratio_.cumsum()
explained_variance_ratio = explained_variance_ratio[:39]

fig, _ = illustrate_nmf_components_for_paper(
    df_V_train_, explained_variance_ratio, df_nmf_models_, pd.Series(BAND_COLS),
    min_explained_variance=MIN_EXPLAINED_VARIANCE, order_components=ORDER_COMPONENTS,
    vmin=vmin, vmax=vmax, xlims=(-1,101), plot_x_ticks=[0, 10, 20, 30, 40, 49]
)

#fig.savefig(os.path.join('figs', 'nmf_exemplary_fold.pdf'), dpi=300, bbox_inches='tight')

We utilize across all folds the same number of components. We choose 5 based on the analysis above.

In [ ]:
N_COMPONENTS = 5
COMPONENT_COLUMNS = list(range(N_COMPONENTS))  # used later
model_folds = [df_nmf_models_[(df_nmf_models_.n_components == N_COMPONENTS)].iloc[0] for df_nmf_models_ in df_nmf_models_folds]

# Offline vibration fingerprint extraction

The process view is simplified in this particular use case.
In contrast to the industrial dataset, in this dataset there are no timestamps. 
However, we know speed and torque for each measurement. 
Therefore, we merge the fingerprint with the operating mode.
Below we plot the weights of a single measurement from the training set.
All measurement directions (x, y and z) are stacked.

In [ ]:
df_W_train_with_OM_folds = []

for i, (model_, df_V_train_, meta_data_train_) in enumerate(zip(model_folds, df_V_train_folds, meta_data_train_folds)):
    W_train_ = model_.W.reshape(-1, N_COMPONENTS)
    df_W_train_ = pd.DataFrame(W_train_)
    #display(f'Fold {i}. Shape: {W_train_.shape}')
    df_W_train_.index = df_V_train_.index
    df_W_train_['direction'] = meta_data_train_['direction']

    # add operating mode (OM)
    df_W_train_with_OM_ = pd.merge(df_W_train_, meta_data_train_.drop(columns=['direction']), left_index=True, right_index=True)
    df_W_train_with_OM_folds.append(df_W_train_with_OM_)

# illustrate weights of single measurement 
rpm=100
torque=500
run=1
fold=0

df_W_train_with_OM_ = df_W_train_with_OM_folds[fold]
df_ = df_W_train_with_OM_[(df_W_train_with_OM_['rotational speed [RPM]']==rpm) &
                          (df_W_train_with_OM_['torque [Nm]']==torque) &
                          (df_W_train_with_OM_['sample_id']==run)]
df_ = df_.set_index('direction')
fig, ax = plt.subplots()
sns.heatmap(df_[list(range(N_COMPONENTS))], annot=True, fmt=".3f", ax=ax, cmap='Blues', vmin=0, vmax=0.1, cbar=False)
ax.set_title(f'Measurement {run} @ {rpm} rpm, {torque} Nm');
ax.set_xlabel('component');

A **context-sensitive vibration fingerprint** is the aggregation over all measurements of a given operating mode.
Below, we plot the context-sensitive vibration fingerprint for the same operating mode. 

In [ ]:
df_W_train_with_OM_ = df_W_train_with_OM_folds[fold]
df_ = df_W_train_with_OM_[(df_W_train_with_OM_['rotational speed [RPM]']==rpm) & (df_W_train_with_OM_['torque [Nm]']==torque)]
df_ = df_[list(range(N_COMPONENTS)) + ['direction']].groupby('direction').mean()
fig, ax = plt.subplots()
sns.heatmap(df_, annot=True, fmt=".3f", ax=ax, cmap='Blues', vmin=0, vmax=0.1, cbar=False)
ax.set_title(f'Vibration fingerprint @ {rpm} rpm, {torque} Nm');
ax.set_xlabel('component');

We observe that the vibration fingerprint and the measurement are very similar, due to at least following reasons: 
- As the measurements were taken in a controlled environment, the operating mode is constant for each operating condition.
- There are very few measurements per operating mode. Therefore, the fingerprint is influenced to a high degree by the single measurement.

The vibration fingerprint serves as key component of the context-sensitive anomaly detection.

In what follows we construct the vibration fingerprint for each operating mode in the training set, where we treat each unique combination of speed and torque as a separate operating mode. We then aggregate the fingerprints for each operating mode. The 73 fingerprints are visualized below. Note that the number of fingerprints may vary depending on the fold, as not all operating modes are present in each fold.

In [ ]:
SHOW_FINGERPRINTS = True   # set SHOW_FINGERPRINTS to False, if visualization should not be shown

# for each unique combination of RPM and torque, assign a unique cluster label
cluster_label_unique_name_mapping_folds = []
for i, df_W_train_with_OM_ in enumerate(df_W_train_with_OM_folds):
    df_W_train_with_OM_['cluster_label_unique'] = df_W_train_with_OM_.groupby(['rotational speed [RPM]', 'torque [Nm]']).ngroup()
    df_W_train_with_OM_folds[i] = df_W_train_with_OM_
    cluster_label_unique_name_mapping_folds.append(df_W_train_with_OM_.groupby('cluster_label_unique').first()[['rotational speed [RPM]', 'torque [Nm]']].reset_index())

# extract operating mode wise fingerprints
grouping_vars = ['direction', 'cluster_label_unique']
fingerprints_folds = []
for i, df_W_train_with_OM_ in enumerate(df_W_train_with_OM_folds):
    df_ = df_W_train_with_OM_[COMPONENT_COLUMNS + grouping_vars].copy()
    fingerprints_ = {
        om: om_group.groupby(['direction']).mean().drop(columns=['cluster_label_unique']) for om, om_group in df_.groupby('cluster_label_unique')
    }
    fingerprints_folds.append(fingerprints_)

# illustrate fingerprints
fingerprints_ = fingerprints_folds[fold]
cluster_label_unique_name_mapping_ = cluster_label_unique_name_mapping_folds[fold]
if SHOW_FINGERPRINTS:
    nrows = math.ceil(len(fingerprints_) / 3)
    fig, axes = plt.subplots(figsize=(15, 2*nrows), nrows=nrows, ncols=3, sharex=True, sharey=True)
    for om, ax in tqdm(zip(fingerprints_, axes.flat), total=len(fingerprints_), desc='Plotting fingerprints'):
        om_group = fingerprints_[om]
        om_group.columns = om_group.columns.astype(str)
        sns.heatmap(om_group, annot=True, fmt=".3f", ax=ax, cmap='Blues', vmin=0, vmax=0.1, cbar=False)
        rpm = cluster_label_unique_name_mapping_[cluster_label_unique_name_mapping_.cluster_label_unique == om]['rotational speed [RPM]'].values[0]
        Nm = cluster_label_unique_name_mapping_[cluster_label_unique_name_mapping_.cluster_label_unique == om]['torque [Nm]'].values[0]
        ax.set_title(f'OM {om}, ({rpm} rpm, {Nm} Nm))')
        ax.set_xlabel('component')
    fig.tight_layout()

---

# Appendix

What follows in here can be seen as additional exploration. 
It is not necessary to understand the anomaly detection work flow.

## Similarity between fingerprints

Below, we visualize the pairwise distance between the fingerprints. We observe that the fingerprints are very similar in some cases, when the operating conditions are similar. For instance, fingerprints 0-4 are similar, as they express the vibrations at 100 rpm.

In [ ]:
pairwise_distances = []
fingerprints_ = fingerprints_folds[fold]
for om1 in fingerprints_:
    fp1 = fingerprints_[om1]
    for om2 in fingerprints_:
        fp2 = fingerprints_[om2]
        dist_ = distance_metrics.cosine_distance(fp1, fp2)
        pairwise_distances.append({'om1': om1, 'om2': om2, 'dist': dist_})
df_pairwise_dist = pd.DataFrame(pairwise_distances)

df_plot = df_pairwise_dist.pivot(columns="om1", index="om2", values="dist")
fig, ax = plt.subplots(figsize=(20, 16))
sns.heatmap(df_plot, ax=ax, cmap='Blues', annot=False, fmt=".2f")
ax.set_title(f"Cosine distance between fingerpints")

## Constructing consensus fingerprints

A possible extension of our approach is to utilize consensus fingerprints.
Consensus fingerprints are constructed by summarizing similar fingerprints by clustering.
This idea was not used in the anomaly detection work flow, but is presented here for completeness.

Below we show the dendrograms for clustering the fingerprints using hierarchical clustering with 4 different linkage methods. The dendrograms illustrate how the fingerprints are clustered based on their similarity. The fingerprints are clustered based on the cosine distance between the fingerprints.

In [ ]:
fingerprints_ = fingerprints_folds[fold]

def plot_dendrogram(linkage_matrix, ax=None, **kwargs):
    dendrogram(linkage_matrix, ax=ax, **kwargs)
    ax.set_title('Hierarchical Clustering Dendrogram')
    ax.set_xlabel('Samples')
    ax.set_ylabel('Distance')
    return ax

fingerprints_feature_space = np.vstack(pd.Series(fingerprints_).apply(lambda df: df.stack().values).to_numpy())

fig, axes = plt.subplots(figsize=(15, 10), ncols=2, nrows=2, sharey=True)
axes = axes.flatten()
fig.suptitle('Hierarchical Clustering Dendrogram')

# Single linkage clustering
linkage_matrix_avg = linkage(pdist(fingerprints_feature_space, metric='cosine'), optimal_ordering=True, method='single')
ax = plot_dendrogram(linkage_matrix_avg, ax=axes[0])
ax.set_title('Single-link (nearest point)')

# Complete linkage clustering
linkage_matrix_ward = linkage(pdist(fingerprints_feature_space, metric='cosine'), optimal_ordering=True, method='complete')
ax = plot_dendrogram(linkage_matrix_ward, ax=axes[1])
ax.set_title('Complete-link (farthest point)')

# Average linkage clustering: WPGMA
linkage_matrix_avg = linkage(pdist(fingerprints_feature_space, metric='cosine'), optimal_ordering=True, method='weighted')
ax = plot_dendrogram(linkage_matrix_avg, ax=axes[2])
ax.set_title('Average-link (WPGMA)')

# Average linkage clustering: UPGMA
linkage_matrix_avg = linkage(pdist(fingerprints_feature_space, metric='cosine'), optimal_ordering=True, method='average')
ax = plot_dendrogram(linkage_matrix_avg, ax=axes[3])
ax.set_ylim(0, 1)
ax.set_title('Average-link (UPGMA): preferred method')

fig.tight_layout()

For instance, if we choose the average linkage method, we can see that the fingerprints are clustered into 4 clusters using a threshold of 0.5.

In [ ]:
# Set a threshold to determine the number of clusters (you can adjust this threshold as needed)
threshold = 0.5

linkage_matrix_avg = linkage(pdist(fingerprints_feature_space, metric='cosine'), optimal_ordering=True, method='average')
fig, ax = plt.subplots(figsize=(20, 7))
ax = plot_dendrogram(linkage_matrix_avg, ax=ax, color_threshold=threshold, above_threshold_color='k')
ax.set_title('Average-link (UPGMA)')
ax.axhline(y=threshold, c='k', ls='--', lw=1)

# Get the cluster labels based on the threshold
cluster_labels = fcluster(linkage_matrix_avg, threshold, criterion='distance')
# generate dictionary where each operating mode is mapped to the respective group and save locally
cluster_labels_dict = dict(zip(list(string.ascii_uppercase[0:len(cluster_labels)]), cluster_labels))
#if CACHE_RESULTS:
if False:
    with open('cluster_labels_dict.pickle', 'wb') as fp:
        pickle.dump(cluster_labels_dict, fp)

A possible clustering of the fingerprints is shown below.
Each row is a fingerprint and each column is a component of the fingerprint (over all three directions).
For instance, the fingerprints 0-4 which were previously shown to be very similar, as they all express vibrations at 100 rpm, would be clustered together.

In [ ]:
tick_color = {1: 'blue', 2: 'red', 3: 'green', 4: 'orange', 5: 'purple', 6: 'brown', 7: 'pink', 8: 'gray', 9: 'olive', 10: 'cyan'}

dfs_ = {om: fingerprints_[om].values.flatten() for om in fingerprints_}
fig, ax = plt.subplots(figsize=(4, 18))
ax.set_title('Fingerprinting featurespace with corresponding cluster', fontsize=32)

df = pd.DataFrame(dfs_).T
sns.heatmap(df, ax=ax, cmap='Blues', annot=False, fmt=".2f", norm=LogNorm())
texts = []
for tick, cluster_label_ in zip(ax.get_yticklabels(), cluster_labels):
    tick.set_color(tick_color[cluster_label_])
    texts.append(f'{tick.get_text()} ({cluster_label_})')
ax.set_xlabel('Component', fontsize=24)
ax.set_ylabel('Fingerprint (cluster-id)', fontsize=24)
ax.set_yticklabels(texts, rotation=0, fontsize=16);
fig.tight_layout()

©, 2023, Sirris